(file-types:notebooks)=
# Les données utilisées

```{contents}
:local:
```

Pour analyser le débat des retraites, nous proposons d'analyser le discours de nos représentants, ici les parlementaires. Pour cela, nous proposons d'analyser
deux sources de données : Twitter et Assemblée Nationale.


## Données sur les parlementaires 

Dans l'analyse des données provenant de nos deux corpus, il est nécessaire d'obtenir différentes informations concernant les parlementaires comme un identifiant unique, à quel groupe appartient-il(elle), a-t-il(elle) un compte sur 
Twitter, etc. Si les données sont rendus par l'administration publique, leur parcours est fastidieux ! Heureusement, les personnes derrières le site [NosDéputés.fr](https://www.nosdeputes.fr/) propose une table synthétique[^nos-deputes-fichier] contenant les informations nécessaires. Pour chaque député, un identifiant (`slug` dans le fichier) est associé à chaque député. Dans les fichiers mises à disposition ici, nous utiliserons ces identifiants.

[^nos-deputes-fichier]: [Source des données en CSV](https://github.com/regardscitoyens/nosdeputes.fr/blob/master/doc/opendata.md) 

## Transcription du débat de la réforme à l'assemblée nationale

Selon l'article 33 de la constitution[^texte-constitution]:

>Les séances des deux assemblées sont publiques. Le compte rendu intégral des débats est publié au Journal officiel.<br><br> Chaque assemblée peut siéger en comité secret à la demande du Premier ministre ou d'un dixième de ses membres.

Depuis 2017, l'ensemble des transcriptions des séances à l'assemblée nationale sont disponible son [site web](https://www.assemblee-nationale.fr/dyn/16/comptes-rendus/seance/). Pour récupérer ces données dans un format exploitable (.csv), plusieurs solutions sont 
envisageables. A l'aide des fichiers `.pdf` de chaque séance, programmer un code permettant d'en extraire les interventions des députés. Problème, il est difficile de séparer les différents blocs de textes (auteur, intervention, etc.) à cause du format du PDF. La deuxième solution est de faire un programme de *web-scrapping* qui récupérera le code HTML[^html-def] des pages webs des séances disponibles sur le site de l'assemblée nationale. 

### HTML in a nutshell 🥜
L'HTML ou HyperText Markup Language est un language permettant de décrire le contenu d'une page web (texte, image, vidéo, etc.), le style (couleur du texte, police de caractères) et plein d'autres choses. Le code suivant permet d'afficher une image suivi d'un texte:

```html
<img src="https://cataas.com/cat/says/meow?size=1" alt="image de chat" id="cat_image" class="image"/>
<p id="legend-image-chat">
    Ceci est un chat ! 
</p>
```
Ce qui donnera le résultat suivant:
<div style="background-color:#aaa;width:300px;padding:1em;">
<img src="https://cataas.com/cat/says/meow?size=1" style="width:100%" alt="image de chat" id="cat_image" class="image"/>
<p id="legend-image-chat" style="color:black;">
    Ceci est un chat ! 
</p>
</div>
<br> 

Un code HTML est composé de balises pour chaque type de contenue que l'on veut afficher. La balise `<img>` permet d'afficher une image, la balise `<p>` permet d'afficher un paragraphe, etc. Dans ces balises, il est possible de rajouter des attributs pour
impacter le style de l'élément (taille, couleur, etc) ou lui assigner des identifiants:  `id=\"..."`  et `class=\"..."`. Ces identifiants permettent de rassembler ou d'isoler certains éléments à l'aide de labels. Un des avantages premier est d'appliquer un même style à plusieurs balises. Par exemple, je veux que tous mes paragraphes `<p>` soit de couleur rouge. Pour cela, j'ajoute un attribut `class="red-paragraph"` à mes balises, puis dans un fichier extérieur (fichier CSS), j'indique que le contenu des balises `<p class="red-paragraph">` sera de couleur rouge comme ceci:

```css
p .red-paragraph{
    color: red;
}

```

Le deuxième avantage, celui qui nous intéresse est de pouvoir permettre à un programme informatique de retrouver les éléments d'une page ! Le code ci-dessous est une extrait de la page contenant la transcription d'une séance publique. En utilisant, les identifiants indiqués par l'attribut `class`, on voit que toutes les transcriptions se trouvent dans le block `<span>` avec l'identifiant `intervention`  qui lui-même se trouve dans un block `<div>` avec l'identifiant `crs-inter-groupe.`

```html
<div class="crs-inter-groupe">
    ...
    <span class="intervention">
        Hier soir, l’Assemblée a poursuivi la discussion des articles, s’arrêtant à l’amendement n<sup>o</sup> 1447 portant article additionnel après l’article 3.
    </span>
</div>
```

Une fois que vous avez compris la mécanique, il suffit d'identifier quelles sont les identifiants contenant :
 * le texte de l'intervention
 * le nom de l'intervenant
 * le nom de la personne qui préside la séance
 * l'organisation de la séance (section, sous-section)
 * la date de la séance
 * etc..


```{note}
Le programme de web-scrapping des séances publiques à l'assemblée nationale est disponible [ici](https://github.com/Jacobe2169/debat_parlement_text_analysis/blob/master/1_get_transcript_seance_assemblee.py).
```


[^texte-constitution]: [Texte intégrale de la constitution de 1958](https://www.conseil-constitutionnel.fr/le-bloc-de-constitutionnalite/texte-integral-de-la-constitution-du-4-octobre-1958-en-vigueur)
[^html-def]: [Qu'est que le HTML ?](https://developer.mozilla.org/fr/docs/Web/HTML) (documentation de Mozilla)


## Collection des tweets des parlementaires (Twitter) 

Une des particularités du débat parlementaires pour la réforme des retraites est sa courte durée. En effet, si une première lecture est effectuée entre le 6 février et le 16 février 2023, l'utilisation du 49.3 ne permettra pas d'autres débat dans l'hémicycle. Le traitement de la réforme des retraites par les parlementaires ne se confine pas au palais Bourbon ! Une majorité de responsable politique utilise les réseaux sociaux pour déclarer leur position en réaction à différents événements. Il est donc intéressant de traiter ces interventions ! Dans le cadre de la réforme des retraites, ces interventions se déroulent sur une plus grande période ! Nous avons choisi de récupérer les tweets de la première lecture du texte au retrait de l'amendement LIOT du 7 juin 2023.

```{warning}
Seulement 17 députés n'ont pas de compte Twitter. **(Source: NosDéputés.fr)**
```

Pour extraire des données provenant de la plateforme Twitter, il est d'usage d'utiliser API[^twitter-api] de cette dernière. Elle permet de faire des requêtes sur la base de données Twitter pour récupérer des informations sur des utilisateurs, des tweets, mais aussi d'appliquer des filtres (date, mots-clés, etc.). Malheureusement, depuis le rachat de Twitter par Elon Musk, l'accès gratuit de l'API est devenu très restreint et les offres payants beaucoup trop onéreuse pour ma simple personne ! Une autre méthode est, vous l'avez deviné, de passer par du web-scrapping ! Comme pour les séances publiques, nous récupérons l'HTML de la page Twitter d'un député, puis l'on extrait ses tweets jusqu'à la date souhaitée ! 

```{note}
Depuis le 1er juillet, il n'est plus possible (ou difficile) de faire du web-scrapping sur Twitter.
<blockquote class="twitter-tweet" data-lang="en" data-dnt="true" data-theme="dark"><p lang="en" dir="ltr">To address extreme levels of data scraping &amp; system manipulation, we’ve applied the following temporary limits:<br><br>- Verified accounts are limited to reading 6000 posts/day<br>- Unverified accounts to 600 posts/day<br>- New unverified accounts to 300/day</p>&mdash; Elon Musk (@elonmusk) <a href="https://twitter.com/elonmusk/status/1675187969420828672?ref_src=twsrc%5Etfw">July 1, 2023</a></blockquote> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script> 
```


```{note}
Le code pour récupérer les tweets est disponible [ici](https://gist.github.com/Jacobe2169/1496ec6517cde2d323bdcbee7da5d460)
```


[^twitter-api]: [Lien vers la documentation de l'API de Twitter](https://developer.twitter.com/en/docs/twitter-api)

